In [ ]:
import torchio as tio
import monai
from monai.transforms import (
    AddChanneld,
    LoadImage,
    LoadImaged,
    Orientationd,
    Rand3DElasticd,
    RandAffined,
    Spacingd,
    ToTensord,
    RandAffine
)
import torchvision as tv
import os
import numpy as np
import custom_models
import torch as t
from copy import deepcopy
import loss_module
import time
import matplotlib.pyplot as plt
from SVR_Preprocessor import Preprocesser
from SVR_outlier_removal import outlier_removal

In [ ]:
pixdim = (2.0,2.0,2.0)

src_folder = "sample_data"
prep_folder = "cropped_images"
src_folder = "sample_data"
result_folder = os.path.join("results","two_stacks_first_outlier_removal")

stack_filenames = ["10_3T_nody_001.nii.gz",
                
                "14_3T_nody_001.nii.gz",
                
                "21_3T_nody_001.nii.gz",
                
                "23_3T_nody_001.nii.gz"]
device = "cpu"
mask_filename = "mask_10_3T_brain_smooth.nii.gz"
mode =  "bilinear"

svr_preprocessor = Preprocesser(src_folder, prep_folder, result_folder, stack_filenames, mask_filename, device, mode)
        
fixed_images, stacks = svr_preprocessor.preprocess_stacks_and_common_vol(pixdim)

tio_image = svr_preprocessor.monai_to_torchio(stacks[0])

In [ ]:
tio_image.save('monai_to_tio.nii.gz')

In [ ]:
transform = tio.Resample((3,3,3))
transformed_tio = transform(tio_image)

In [ ]:
stack = stacks[0]
stack_updated = svr_preprocessor.update_monai_from_tio(transformed_tio,stack)

In [ ]:
nifti_saver = monai.data.NiftiSaver(output_dir="torchio_to_monai", output_postfix=f"{0:02}",
                                            resample=False, mode="bilinear", padding_mode="zeros",
                                            separate_folder=False)
nifti_saver.save(stack_updated["image"],stack_updated["image_meta_dict"])

In [ ]:
transformed_tio.save("torchio_to_monai/original.nii.gz")

In [ ]:
affine_grid = t.nn.functional.affine_grid(t.rand(1,3,4),(1,1,10,10,5))

In [ ]:
stack = stacks[0]

In [ ]:
tio_image = tio.Image(tensor=stack["image"], affine= stack["image_meta_dict"]["affine"])

In [ ]:
tio_image.tensor.shape

In [ ]:
tio_image.affine

In [ ]:
stack["image_meta_dict"]["spatial_shape"]

In [ ]:
np.array(list(tio_image.tensor.shape)[1:])

In [ ]:
tio_image_2 = svr_preprocessor.monai_to_torchio(stack)